In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# Count number of files in test set
import os
from pathlib import Path
from globals import TEST_DS_PATH

# Count number of files in test set
def count_files(path):
    return len(list(Path(path).glob("*")))

n_normal = count_files(f"{TEST_DS_PATH}/normal")
n_malignant = count_files(f"{TEST_DS_PATH}/malignant")
n_benign = count_files(f"{TEST_DS_PATH}/benign")

print("normal", n_normal)
print("malignant", n_malignant)
print("benign", n_benign)


normal 31
malignant 39
benign 86


In [20]:
import globals
import os
import pandas as pd

# Loads confusion matrices from each CV, calculates the mean and standard deviation
# and saves them to a file

RANDOM_INIT_EXP_NAMES = [f"noimagenet-random-{seed}" for seed in globals.SEEDS]
paths = [os.path.join(globals.BASE_PATH, "experiments", n, f"conf_matrix_{n}.csv") for n in RANDOM_INIT_EXP_NAMES]

conf_matrices = [pd.read_csv(path, index_col=0) for path in paths]

mean_conf_matrix = pd.concat(conf_matrices).groupby(level=0).mean()
std_conf_matrix = pd.concat(conf_matrices).groupby(level=0).std()

mean_conf_matrix

,benign,malignant,normal
benign,0.800000,0.105814,0.094186
malignant,0.176923,0.784615,0.038462
normal,0.116129,0.035484,0.848387


In [21]:
std_conf_matrix

,benign,malignant,normal
benign,0.048037,0.042264,0.024788
malignant,0.067786,0.061870,0.045629
normal,0.064874,0.035500,0.066371


In [22]:
overall_mean_acc = (n_normal * mean_conf_matrix['normal']['normal'] + n_malignant * mean_conf_matrix['malignant']['malignant'] + n_benign * mean_conf_matrix['benign']['benign']) / (n_normal + n_malignant + n_benign)
overall_std_acc = (n_normal * std_conf_matrix['normal']['normal'] + n_malignant * std_conf_matrix['malignant']['malignant'] + n_benign * std_conf_matrix['benign']['benign']) / (n_normal + n_malignant + n_benign)

print("Overall mean accuracy:", overall_mean_acc)
print("Overall std accuracy:", overall_std_acc)

name = "nofinetune"

mean_conf_matrix.to_csv(f"../results/{name}_mean_conf_matrix.csv")
std_conf_matrix.to_csv(f"../results/{name}_std_conf_matrix.csv")

with open(f"../results/{name}_mean_acc.txt", "w") as f:
    f.write(str(overall_mean_acc))

with open(f"../results/{name}_std_acc.txt", "w") as f:
    f.write(str(overall_std_acc))

Overall mean accuracy: 0.8057692307692307
Overall std accuracy: 0.05513857025856765
